In [ ]:
import os
import json
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from google.colab import drive
import tensorflow as tf
import cv2


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile
# import os

# # Path to the ZIP file
# zip_file_path = '/content/drive/My Drive/2020-02-14_InfraredSolarModules.zip'

# # Extract the ZIP file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/My Drive/extracted_files')

# # Access the extracted files
# extracted_images_dir = '/content/drive/My Drive/extracted_files/images'
# metadata_path = '/content/drive/My Drive/extracted_files/module_metadata.json'


In [ ]:
image_dir = '/content/drive/My Drive/extracted_files/InfraredSolarModules'
metadata_path = '/content/drive/My Drive/extracted_files/InfraredSolarModules/module_metadata.json'


In [ ]:
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Prepare lists for images and labels
image_paths = []
labels = []

for image_id, info in metadata.items():
    image_paths.append(os.path.join(image_dir, info['image_filepath']))
    labels.append(info['anomaly_class'])

from collections import Counter

# Count the number of occurrences of each label
label_counts = Counter(labels)

# Print the count for each label
for label, count in label_counts.items():
    print(f"Label: {label}, Count: {count}")


# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded)


Label: No-Anomaly, Count: 10000
Label: Cell, Count: 1877
Label: Hot-Spot, Count: 249
Label: Offline-Module, Count: 827
Label: Vegetation, Count: 1639
Label: Diode, Count: 1499
Label: Shadowing, Count: 1056
Label: Cracking, Count: 940
Label: Diode-Multi, Count: 175
Label: Hot-Spot-Multi, Count: 246
Label: Cell-Multi, Count: 1288
Label: Soiling, Count: 204


In [ ]:
# import concurrent.futures
# import cv2
# import numpy as np
# i=0

# def load_and_preprocess_image(path):
#     image = cv2.imread(path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     # image = cv2.resize(image, (224, 224))  # Resize to a larger size
#     # image_array = image / 255.0
#     return image


# with concurrent.futures.ProcessPoolExecutor() as executor:
#     images = list(executor.map(load_and_preprocess_image, image_paths))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
# images_array = np.array(images)
# np.save('processed_images.npy', images_array)

In [ ]:
images_array = np.load('/content/processed_images.npy')

In [ ]:
resized_images = np.zeros((len(images_array), 24, 40, 3), dtype=np.float32)

# Resize images and normalize them
for i in range(len(images_array)):
    resized_images[i] = cv2.resize(images_array[i], (40, 24))  # Resize to (width, height)
    resized_images[i] /= 255.0  # Normalize the image

X_train, X_test, y_train, y_test = train_test_split(resized_images, labels_one_hot, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(24, 40, 3)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)  # Try different learning rates
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
)


Epoch 1/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 15s 33ms/step - accuracy: 0.4841 - loss: 2.1556 - val_accuracy: 0.4875 - val_loss: 1.8152
Epoch 2/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.4954 - loss: 1.7835 - val_accuracy: 0.4875 - val_loss: 1.7382
Epoch 3/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.5050 - loss: 1.7071 - val_accuracy: 0.5716 - val_loss: 1.4562
Epoch 4/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.5858 - loss: 1.3790 - val_accuracy: 0.6438 - val_loss: 1.1962
Epoch 5/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.6546 - loss: 1.1556 - val_accuracy: 0.6687 - val_loss: 1.0786
Epoch 6/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.6754 - loss: 1.0841 - val_accuracy: 0.6831 - val_loss: 1.0395
Epoch 7/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.6832 - loss: 1.0234 - val_accuracy: 0.6872 - val_loss: 1.0025
Epoch 8/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.6970 - loss: 0.9513 - val_acc

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

accuracy_score(y_pred_classes,y_true_classes)

125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


0.7675

##Second Model


In [ ]:


from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load DenseNet121 model pre-trained on ImageNet without the top classification layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

base_model.trainable = False


x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Check the model summary
# model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
)

Epoch 1/30
400/400 ━━━━━━━━━━━━━━━━━━━━ 34s 32ms/step - accuracy: 0.5387 - loss: 1.5769 - val_accuracy: 0.6472 - val_loss: 1.1638
Epoch 2/30
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6405 - loss: 1.1970 - val_accuracy: 0.6653 - val_loss: 1.0739
Epoch 3/30
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6556 - loss: 1.1193 - val_accuracy: 0.6809 - val_loss: 1.0162
Epoch 4/30
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6780 - loss: 1.0335 - val_accuracy: 0.6819 - val_loss: 1.0021
Epoch 5/30
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6842 - loss: 1.0150 - val_accuracy: 0.6897 - val_loss: 0.9940
Epoch 6/30
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6958 - loss: 0.9779 - val_accuracy: 0.6928 - val_loss: 0.9721
Epoch 7/30
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6968 - loss: 0.9663 - val_accuracy: 0.6909 - val_loss: 0.9891
Epoch 8/30
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.6982 - loss: 0.9561 - val_acc

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

accuracy_score(y_pred_classes,y_true_classes)

125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step


0.721